In [ ]:
import torch 
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data import Dataset ,DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm

In [ ]:
class PlayingCardDataset(Dataset):
    def __init__(self,data_dir, transform = None):
        self.data =ImageFolder(data_dir,transform = transform)
    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.data[idx]

    def classes(self):
        return self.data.classes

In [ ]:
dataset = PlayingCardDataset(data_dir='/kaggle/input/cards-image-datasetclassification/train')

In [ ]:
len(dataset)

In [ ]:
image, label = dataset[6000]
image


In [ ]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
])

data_dir ='/kaggle/input/cards-image-datasetclassification/train'
dataset = PlayingCardDataset(data_dir,transform)

In [ ]:
image , label = dataset[0]
image.shape

*DataLoaders*

In [ ]:
dataloader = DataLoader(dataset, batch_size=32, shuffle= True)


In [ ]:
for images, labels in dataloader:
    break

In [ ]:
images.shape , labels.shape

In [ ]:
class SimpleCardClassifier(nn.Module):
    def __init__(self,num_classes=53):
        super(SimpleCardClassifier , self).__init__()
        self.base_model = timm.create_model('efficientnet_b0',pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])       
        enet_out_size = 1280
        self.classifier = nn.Linear(enet_out_size, num_classes)

    def forward(self , x):
        x= self.features(x)
        output = self.classifier(x)
        return output

In [ ]:
model = SimpleCardClassifier(num_classes=53)

In [ ]:
example_out = model(images)
example_out.shape

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr =0.001)

In [ ]:
criterion(example_out ,labels)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

train_folder = '../input/cards-image-datasetclassification/train/'
valid_folder = '../input/cards-image-datasetclassification/valid/'
test_folder = '../input/cards-image-datasetclassification/test/'

train_dataset = PlayingCardDataset(train_folder, transform = transform)
valid_dataset = PlayingCardDataset(valid_folder, transform = transform)
test_dataset = PlayingCardDataset(test_folder, transform = transform)

train_loader = DataLoader(train_dataset , batch_size = 32 , shuffle = True)
val_loader = DataLoader(valid_dataset , batch_size = 32 , shuffle = False)
test_loader = DataLoader(test_dataset , batch_size = 32 , shuffle = False)

In [ ]:
num_epoch = 5
train_loss , val_losses = [],[]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = SimpleCardClassifier(num_classes=53)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc='Training loop'):
        # Move inputs and labels to the device
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)
    train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(train_loss)
    
    # Validation phase
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc='Validation loop'):
            # Move inputs and labels to the device
            images, labels = images.to(device), labels.to(device)
         
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * labels.size(0)
    val_loss = running_loss / len(val_loader.dataset)
    val_losses.append(val_loss)
    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}, Validation loss: {val_loss}")



In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.show()

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# Load and preprocess the image
def preprocess_image(image_path, transform):
    image = Image.open(image_path).convert("RGB")
    return image, transform(image).unsqueeze(0)

# Predict using the model
def predict(model, image_tensor, device):
    model.eval()
    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
    return probabilities.cpu().numpy().flatten()

# Visualization
def visualize_predictions(original_image, probabilities, class_names):
    fig, axarr = plt.subplots(1, 2, figsize=(14, 7))
    
    # Display image
    axarr[0].imshow(original_image)
    axarr[0].axis("off")
    
    # Display predictions
    axarr[1].barh(class_names, probabilities)
    axarr[1].set_xlabel("Probability")
    axarr[1].set_title("Class Predictions")
    axarr[1].set_xlim(0, 1)

    plt.tight_layout()
    plt.show()

# Example usage
test_image = "/kaggle/input/cards-image-datasetclassification/test/five of diamonds/2.jpg"
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

original_image, image_tensor = preprocess_image(test_image, transform)
probabilities = predict(model, image_tensor, device)

# Assuming dataset.classes gives the class names
class_names = dataset.classes 
visualize_predictions(original_image, probabilities, class_names)